In [1]:
import math
import numpy as np
%pylab inline
%run CreatePitzerV3.ipynb

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib
[[ 1.43783204e+01  5.60767400e-03 -4.22185236e+02 -2.51226677e+00
   0.00000000e+00 -2.61718135e-06  4.43854508e+00 -1.70502337e+00]
 [-4.83060685e-01  1.40677470e-03  1.19311989e+02  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.23433299e+00]
 [-1.00588714e-01 -1.80529413e-05  8.61185543e+00  1.24880950e-02
   0.00000000e+00  3.41172108e-08  6.83040995e-02  2.93922611e-01]]
[-1.3764e+01  1.2121e-01 -2.7642e-04  0.0000e+00 -2.1515e-01 -3.2743e+01]


In [2]:
# binaryV3     This function calculates  binary interaction parameters 
#              [beta_0, beta_1, beta_2, C_phi] at a given temperature 
#              and produces arrays to store them. For HSO4, two additional
#              parameters are required [C0_HSO4, C1_HSO4]. It makes use of 
#              coefficients provided in PitzerV3.mat. Alternatively, these 
#              coefficients can be found in "CreatePitzerV3.m" file.
#   
#              These interaction parameters are needed to calculate free activity 
#              coefficients using Pitzer equations 
#                              *******************************
#              [beta_0, beta_1, beta_2, C_phi, C0_HSO4, C1_HSO4] = binaryV3(T)
#                             T = Temperature in Kelvin
#                             NB:- one value of T at a time. 
#                              *******************************
#              See also  binaryV3,  ternaryV3(T), gammaFV3
##              NUMBERING SYSTEM
#               Major Cations    
#               Na = 0;       K = 1;     Mg = 2;       Ca = 3;      Sr = 4
#
#               ************************************************************************************
#
#               Minor Cations
#                H = 5;      Li = 6;      Rb = 7;       Cs = 8;      NH4 = 9;    Ba = 10;   Mn = 11;    
#           Fe(II)= 12;     Co = 13;     Ni = 14;   Cu(II)= 15;      Zn = 16;   UO2 = 17;   Be = 18;    
#              Cd = 19;  Pb(II)= 20;   Cu(I)= 21;      La = 22;      Ce = 23;    Pr = 24;   Nd = 25;      
#              Pm = 26;     Sm = 27;     Eu = 28;      Gd = 29;      Tb = 30;    Dy = 31;   Ho = 32;      
#              Er = 33;     Tm = 34;     Yb = 35;      Lu = 36;       Y = 37;    Al = 38;   Ga = 39;      
#              In = 40; Fe(III)= 41;
#
#               ************************************************************************************
#
#               Major Anions   
#               Cl = 0;     SO4 = 1;     CO3 = 2;     HCO3 = 3;    Br = 4;     F = 5   B(OH)4 = 6
# 
#               ************************************************************************************
#
#              Minor Anions
#                  OH = 7;   HSO4 = 8;     HS = 9;      I = 10;    ClO3  = 11;  ClO4 = 12;    BrO3 = 13;     
#                CNS = 14;   NO2 = 15;   NO3 = 16;  H2PO4 = 17;     HPO4 = 18;   PO4 = 19;  H2AsO4 = 20;  
#              HAsO4 = 21;  AsO4 = 22;  HSO3 = 23;    SO3 = 24;  Acetate = 25;
#          
#               ************************************************************************************
#            
#              See also  tenaryV3, gammaFV3

## ---------------------------------------------------------------------------

##  Assign values to global variables
mjCaMax = 5            #In python array goes 0,1,2,3,4,5,6, so mjCaMax must go to 7 to include the 6th variable
mnCaMax = 36
CaMax = mjCaMax + mnCaMax

mjAnMax = 7            #In python array goes 0,1,2,3,4,5,6,7,8 so mjCaMax must go to 8 to include the 7th variable
mnAnMax = 18
AnMax = mjAnMax + mnAnMax

NeutMax = 7

## ------------------------------------------------------------------------------
def binaryV3(T): 
    
    beta_0  = np.zeros([42, 26])  
    beta_1  = np.zeros([42, 26]) 
    beta_2  = np.zeros([42, 26])
    C_phi   = np.zeros([42, 26]) 
    
    def TableA1(T, a):
        # For parameters in Table A1      .....  (Møller, 1988; Millero & Pierrot, 1998)
        Xa = a[0,0] + (a[0,1]*T) + (a[0,2]/T) + (a[0,3]*log(T)) + (a[0,4]/(T-263)) + (a[0,5]*(T**2)) + (a[0,6]/(680-T)) + (a[0,7]/(T-227))
        Ya = a[1,0] + (a[1,1]*T) + (a[1,2]/T) + (a[1,3]*log(T)) + (a[1,4]/(T-263)) + (a[1,5]*(T**2)) + (a[1,6]/(680-T)) + (a[1,7]/(T-227))
        Za = a[2,0] + (a[2,1]*T) + (a[2,2]/T) + (a[2,3]*log(T)) + (a[2,4]/(T-263)) + (a[2,5]*(T**2)) + (a[2,6]/(680-T)) + (a[2,7]/(T-227))
        return Xa, Ya, Za

    def TableA2_1(T, b):
        # For MgCl2 in Table A2           .....  (Pabalan & Pitzer, 1987; Millero & Pierrot, 1998) 
        Xb = b[0,0] + b[0,1]*T + b[0,2]*(T**2)
        Yb = b[1,0] + b[1,1]*T + b[1,2]*(T**2)
        Zb = b[2,0] + b[2,1]*T + b[2,2]*(T**2)
        return Xb, Yb, Zb

    def TableA2_2(T, c):
        # For MgSO42 in Table A2          .....  (Pabalan & Pitzer, 1987; Millero & Pierrot, 1998) 
        Xc = c[0,0]*((T/2)+((298**2)/(2*T))-298) + c[0,1]*(((T**2)/6)+((298**3)/(3*T))-((298**2)/2)) + c[0,2]*(((T**3)/12) + ((298**4)/(4*T))-((298**3)/3))+ c[0,3]*(((T**4)/20)+((298**5)/(5*T))-((298**4)/4)) + c[0,4]*(298-((298**2)/T)) + c[0,5]
        Yc = c[1,0]*((T/2)+((298**2)/(2*T))-298) + c[1,1]*(((T**2)/6)+((298**3)/(3*T))-((298**2)/2)) + c[1,2]*(((T**3)/12) + ((298**4)/(4*T))-((298**3)/3))+ c[1,3]*(((T**4)/20)+((298**5)/(5*T))-((298**4)/4)) + c[1,4]*(298-((298**2)/T)) + c[1,5]
        Zc = c[2,0]*((T/2)+((298**2)/(2*T))-298) + c[2,1]*(((T**2)/6)+((298**3)/(3*T))-((298**2)/2)) + c[2,2]*(((T**3)/12) + ((298**4)/(4*T))-((298**3)/3))+ c[2,3]*(((T**4)/20)+((298**5)/(5*T))-((298**4)/4)) + c[2,4]*(298-((298**2)/T)) + c[2,5]
        return Xc, Yc, Zc 

    def TableA3to6(T, d):
        # For parameters in Table A3 to A6                 
        Xd = d[0,0] + d[0,1]*(T-298.15) + d[0,2]*((T-298.15)**2)
        Yd = d[1,0] + d[1,1]*(T-298.15) + d[1,2]*((T-298.15)**2)
        Zd = d[2,0] + d[2,1]*(T-298.15) + d[2,2]*((T-298.15)**2)
        return Xd, Yd, Zd

    def TableA3A4(T,e):
        # For NaHS, Na2SO3, NaHSO3 in Table A3 and KHS in A4    
        Xe = e[0,0] + e[0,1]/T
        Ye = e[1,0] + e[1,1]/T
        Ze = e[2,0] + e[2,1]/T    
        return Xe, Ye, Ze

    def TableA7(T, f):
        # For parameters in Table A7 and other
        Xf = f[0,0] + (f[0,1] * ((298.15**3)/3 - (298.15**2)*f[0,2]) * (1/T-1/298.15)) + (f[0,1]/6 * (T**2-298.15**2))
        Yf = f[1,0] + (f[1,1] * ((298.15**3)/3 - (298.15**2)*f[1,2]) * (1/T-1/298.15)) + (f[1,1]/6 * (T**2-298.15**2))
        Zf = f[2,0] + (f[2,1] * ((298.15**3)/3 - (298.15**2)*f[2,2]) * (1/T-1/298.15)) + (f[2,1]/6 * (T**2-298.15**2))
        return Xf, Yf, Zf  

    def Eq_HCl(T, g):
        # For HCL in Table A9   
        Xg = g[0,0] + g[0,1]*T + g[0,2]/T
        Yg = g[1,0] + g[1,1]*T + g[1,2]/T
        Zg = g[2,0] + g[2,1]*T + g[2,2]/T
        return Xg, Yg, Zg

    def Eq_HSO4_C94(T, m):
        Xm = m[0,0] + (T-328.15) * (1e-3 * m[0,1]+(T-328.15) * ((1e-3 * m[0,2]/2)+(T-328.15)*1e-3 * m[0,3]/6))
        Ym = m[1,0] + (T-328.15) * (1e-3 * m[1,1]+(T-328.15) * ((1e-3 * m[1,2]/2)+(T-328.15)*1e-3 * m[1,3]/6))
        Zm = m[2,0] + (T-328.15) * (1e-3 * m[2,1]+(T-328.15) * ((1e-3 * m[2,2]/2)+(T-328.15)*1e-3 * m[2,3]/6))
        return Xm, Ym, Zm

    def Eq_REE(T, n):
        # For  REE parameters 
        Xn = n[0,0] + ((298.15**3 * n[0,2])/3  - (n[0,1] * 298.15**2)) * (1 / T - 1 /298.15) + ((n[0,2]/6)*(T**2 - 298.15**2))
        Yn = n[1,0] + ((298.15**3 * n[1,2])/3  - (n[1,1] * 298.15**2)) * (1 / T - 1 /298.15) + ((n[1,2]/6)*(T**2 - 298.15**2))
        Zn = n[2,0] + ((298.15**3 * n[2,2])/3  - (n[2,1] * 298.15**2)) * (1 / T - 1 /298.15) + ((n[2,2]/6)*(T**2 - 298.15**2))
        return Xn, Yn, Zn

    def NoT(o):
        Xo=o[0]
        Yo=o[1]
        Zo=o[2]
        return Xo, Yo, Zo

    def Eq_C0C1(T, p):
        Xp = p[0,0] + (T-328.15) * (1e-3 * p[0,1]+(T-328.15) * ((1e-3 * p[0,2]/2)+(T-328.15)*1e-3 * p[0,3]/6))
        Yp = p[1,0] + (T-328.15) * (1e-3 * p[1,1]+(T-328.15) * ((1e-3 * p[1,2]/2)+(T-328.15)*1e-3 * p[1,3]/6))
        return Xp, Yp

    def M_BOH4(T, q):
        # For beta2 parameters: MgBOH42, CaBOH42, SrBOH42   
        Q = q[0] + q[1]*(T-298.15) + q[2]*((T-303.15)**2)
        return Q 

    def Eq_b2_CaSO4(T, r):
        R = r[0] + (r[1]*T)
        return R

    def Eq_b2_MgSO4(T, s):
        S = s[0]*((T/2)+((298**2)/(2*T))-298) + s[1]*(((T**2)/6)+((298**3)/(3*T))-((298**2)/2)) + s[2]*(((T**3)/12) + ((298**4)/(4*T))-((298**3)/3))+ s[3]*(((T**4)/20)+((298**5)/(5*T))-((298**4)/4)) + s[4]*(298-((298**2)/T)) + s[5]
        return S 

    def Eq_b2_XSO4(T, w):
        W = w[0] + w[1]*(T - 298.15) + w[2]*((T-298.15)**2)
        return W

    # ##############################################################################################################
    #  function P = Eq_CaSO4_Spe (T, p)
    #  P = p(:,1) + p(:,2)*T + p(:,3)*(T^2) + p(:,4)*(T^3) + p(:,5)/T + p(:,6)*log(T); 
    #  end
    # ################################################################################################################# 
    
## Major Cation -  Anion
## Na
    [beta_0[0,0], beta_1[0,0], C_phi[0,0]]    = TableA1(T,param_NaCl)           # NaCl           (0,0)
    [beta_0[0,1], beta_1[0,1], C_phi[0,1]]    = TableA1(T, param_Na2SO4_Moller) # Na2SO4         (0,1)
    [beta_0[0,2], beta_1[0,2], C_phi[0,2]]    = TableA3to6(T,param_Na2CO3)      # Na2CO3         (0,2)
    [beta_0[0,3], beta_1[0,3], C_phi[0,3]]    = TableA3to6(T,param_NaHCO3)      # NaHCO3         (0,3)
    [beta_0[0,4], beta_1[0,4], C_phi[0,4]]    = TableA7(T,param_NaBr)           # NaBr           (0,4)
    [beta_0[0,5], beta_1[0,5], C_phi[0,5]]    = TableA7(T,param_NaF)            # NaF            (0,5)
    [beta_0[0,6], beta_1[0,6], C_phi[0,6]]    = TableA3to6(T,param_NaBOH4)      # NaB(OH)4       (0,6)
    [beta_0[0,7], beta_1[0,7], C_phi[0,7]]    = NoT(param_NaOH)                 # NaOH           (0,7)
    [beta_0[0,8], beta_1[0,8], C_phi[0,8]]    = TableA3to6(T,param_NaHSO4)      # NaHSO4         (0,8)
    [beta_0[0,9], beta_1[0,9], C_phi[0,9]]    = TableA3A4(T, param_NaHS)        # NaHS           (0,9)
    [beta_0[0,10], beta_1[0,10], C_phi[0,10]] = TableA7(T,param_NaI)            # NaI            (0,10)
    [beta_0[0,11], beta_1[0,11], C_phi[0,11]] = TableA7(T,param_NaClO3)         # NaClO3         (0,11)
    [beta_0[0,12], beta_1[0,12], C_phi[0,12]] = TableA7(T,param_NaClO4)         # NaClO4         (0,12)
    [beta_0[0,13], beta_1[0,13], C_phi[0,13]] = TableA7(T,param_NaBrO3)         # NaBrO3         (0,13)
    [beta_0[0,14], beta_1[0,14], C_phi[0,14]] = TableA3to6(T,param_NaCNS)       # NaCNS          (0,14)
    [beta_0[0,15], beta_1[0,15], C_phi[0,15]] = NoT(param_NaNO2)                # NaNO2          (0,15)
    [beta_0[0,16], beta_1[0,16], C_phi[0,16]] = TableA7(T,param_NaNO3)          # NaNO3          (0,16)
    [beta_0[0,17], beta_1[0,17], C_phi[0,17]] = NoT(param_NaH2PO4)              # NaH2PO4        (0,17)
    [beta_0[0,18], beta_1[0,18], C_phi[0,18]] = NoT(param_Na2HPO4)              # Na2HPO4        (0,18)
    [beta_0[0,19], beta_1[0,19], C_phi[0,19]] = NoT(param_Na3PO4)               # Na3PO4         (0,19)
    [beta_0[0,20], beta_1[0,20], C_phi[0,20]] = NoT(param_NaH2AsO4)             # NaH2AsO4       (0,20)
    [beta_0[0,21], beta_1[0,21], C_phi[0,21]] = NoT(param_Na2HAsO4)             # Na2HAsO4       (0,21)
    [beta_0[0,22], beta_1[0,22], C_phi[0,22]] = NoT(param_Na3AsO4)              # Na3AsO4        (0,22)
    [beta_0[0,23], beta_1[0,23], C_phi[0,23]] = TableA3A4(T,param_NaHSO3)       # NaHSO3         (0,23)
    [beta_0[0,24], beta_1[0,24], C_phi[0,24]] = TableA3A4(T,param_Na2SO3)       # Na2SO3         (0,24)
    [beta_0[0,25], beta_1[0,25], C_phi[0,25]] = NoT(param_NaAcetate)            # NaAcetate      (0,25)
    
## K
    [beta_0[1,0], beta_1[1,0], C_phi[1,0]]    = TableA1(T,param_KCl)            # KCl            (1,0) 
    [beta_0[1,1], beta_1[1,1], C_phi[1,1]]    = TableA1(T, param_K2SO4)         # K2SO4          (1,1)
    [beta_0[1,2], beta_1[1,2], C_phi[1,2]]    = TableA3to6(T,param_K2CO3)       # K2CO3          (1,2)    
    [beta_0[1,3], beta_1[1,3], C_phi[1,3]]    = TableA3to6(T,param_KHCO3)       # KHCO3          (1,3)
    [beta_0[1,4], beta_1[1,4], C_phi[1,4]]    = TableA7(T,param_KBr)            # KBr            (1,4)
    [beta_0[1,5], beta_1[1,5], C_phi[1,5]]    = TableA7(T,param_KF)             # KF             (1,5)
    [beta_0[1,6], beta_1[1,6], C_phi[1,6]]    = TableA3to6(T,param_KBOH4)       # KB(OH)4        (1,6)
    [beta_0[1,7], beta_1[1,7], C_phi[1,7]]    = TableA7(T,param_KOH)            # KOH            (1,7)
    [beta_0[1,8], beta_1[1,8], C_phi[1,8]]    = TableA3to6(T,param_KHSO4_MIAMI) # KHSO4          (1,8)
    [beta_0[1,9], beta_1[1,9], C_phi[1,9]]    = TableA3A4(T, param_KHS)         # KHS            (1,9)
    [beta_0[1,10], beta_1[1,10], C_phi[1,10]] = TableA7(T,param_KI)             # KI             (1,10)
    [beta_0[1,11], beta_1[1,11], C_phi[1,11]] = TableA7(T,param_KClO3)          # KClO3          (1,11)
# N/A                                                                           # KClO4          (1,12)
    [beta_0[1,13], beta_1[1,13], C_phi[1,13]] = TableA7(T,param_KBrO3)          # KBrO3          (1,13)
    [beta_0[1,14], beta_1[1,14], C_phi[1,14]] = TableA3to6(T,param_KCNS)        # KCNS           (1,14)
    [beta_0[1,15], beta_1[1,15], C_phi[1,15]] = NoT(param_KNO2)                 # KNO2           (1,15)
    [beta_0[1,16], beta_1[1,16], C_phi[1,16]] = TableA7(T,param_KNO3)           # KNO3           (1,16)
    [beta_0[1,17], beta_1[1,17], C_phi[1,17]] = TableA3to6(T,param_KH2PO4)      # KH2PO4         (1,17)
    [beta_0[1,18], beta_1[1,18], C_phi[1,18]] = NoT(param_K2HPO4)               # K2HPO4         (1,18)
    [beta_0[1,19], beta_1[1,19], C_phi[1,19]] = NoT(param_K3PO4)                # K3PO4          (1,19)
    [beta_0[1,20], beta_1[1,20], C_phi[1,20]] = NoT(param_KH2AsO4)              # KH2AsO4        (1,20)
    [beta_0[1,21], beta_1[1,21], C_phi[1,21]] = NoT(param_K2HAsO4)              # K2HAsO4        (1,21)
# N/A                                                                           # K3AsO4         (1,22) 
# N/A                                                                           # KHSO3          (1,23)
# N/A                                                                           # K2SO3          (1,24)
    [beta_0[1,25], beta_1[1,25], C_phi[1,25]] = NoT(param_KAcetate)             # KAcetate       (1,25) 
    
## Mg
    [beta_0[2,0], beta_1[2,0], C_phi[2,0]]    = TableA2_1(T,param_MgCl2)        # MgCl2          (2,0)
    [beta_0[2,1], beta_1[2,1], C_phi[2,1]]    = TableA2_2(T,param_MgSO4)        # MgSO4          (2,1)
#  N/A                                                                          # MgCO3          (2,2)
    [beta_0[2,3], beta_1[2,3], C_phi[2,3]]    = NoT(param_MgHCO32)              # MgHCO32        (2,3)
    [beta_0[2,4], beta_1[2,4], C_phi[2,4]]    = TableA3to6(T,param_MgBr2)       # MgBr2          (2,4)
# N/A                                                                           # MgF2           (2,5)
    [beta_0[2,6], beta_1[2,6], C_phi[2,6]]    = TableA3to6(T,param_MgBOH42)     # MgB(OH)42      (2,6)
# N/A                                                                           # MgOH2          (2,7)
    [beta_0[2,8], beta_1[2,8], C_phi[2,8]]    = TableA3to6(T,param_MgHSO42)     # MgHSO42        (2,8)
    [beta_0[2,9], beta_1[2,9], C_phi[2,9]]    = NoT(param_MgHS2)                # MgHS2          (2,9)
    [beta_0[2,10], beta_1[2,10], C_phi[2,10]] = NoT(param_MgI2)                 # MgI2           (2,10)
    [beta_0[2,11], beta_1[2,11], C_phi[2,11]] = TableA2_1(T,param_MgCl2)        # MgClO32        (2,11)
    [beta_0[2,12], beta_1[2,12], C_phi[2,12]] = TableA3to6(T,param_MgClO42)     # MgClO42        (2,12)
    [beta_0[2,13], beta_1[2,13], C_phi[2,13]] = TableA2_1(T,param_MgCl2)        # MgBrO32        (2,13)
    [beta_0[2,14], beta_1[2,14], C_phi[2,14]] = TableA2_1(T,param_MgCl2)        # MgCNS2         (2,14)
    [beta_0[2,15], beta_1[2,15], C_phi[2,15]] = TableA2_1(T,param_MgCl2)        # MgNO22         (2,15)
    [beta_0[2,16], beta_1[2,16], C_phi[2,16]] = TableA7(T,param_MgNO32)         # MgNO32         (2,16)
    [beta_0[2,17], beta_1[2,17], C_phi[2,17]] = TableA2_1(T,param_MgCl2)        # MgH2PO42       (2,17)
# N/A                                                                           # MgHPO4         (2,18)
# N/A                                                                           # Mg3(PO4)2      (2,19)
    [beta_0[2,20], beta_1[2,20], C_phi[2,20]] = TableA2_1(T,param_MgCl2)        # MgH2AsO42      (2,20)
# N/A;                                                                          # MgHAsO4        (2,21)
# N/A;                                                                          # Mg3(AsO4)2     (2,22)
    [beta_0[2,23], beta_1[2,23], C_phi[2,23]] = NoT(param_MgHSO32)              # MgHSO32        (2,23)
    [beta_0[2,24], beta_1[2,24], C_phi[2,24]] = NoT(param_MgSO3)                # MgSO3          (2,24)
# N/A;                                                                          # MgAcetate      (2,25)

## Ca
    [beta_0[3,0], beta_1[3,0], C_phi[3,0]]    = TableA1(T,param_CaCl2)          # CaCl2          (3,0)
    [beta_0[3,1], beta_1[3,1], C_phi[3,1]]    = TableA1(T,param_CaSO4)          # CaSO4          (3,1)
# N/A;                                                                          # CaCO3          (3,2)
    [beta_0[3,3], beta_1[3,3], C_phi[3,3]]    = NoT(param_CaHCO32)              # CaHCO32        (3,3)
    [beta_0[3,4], beta_1[3,4], C_phi[3,4]]    = TableA3to6(T,param_CaBr2)       # CaBr2          (3,4)
# N/A                                                                           # CaF2           (3,5)
    [beta_0[3,6], beta_1[3,6], C_phi[3,6]]    = TableA3to6(T,param_CaBOH42)     # CaB(OH)42      (3,6)
    [beta_0[3,7], beta_1[3,7], C_phi[3,7]]    = NoT(param_CaOH2)                # CaOH2          (3,7)
    [beta_0[3,8], beta_1[3,8], C_phi[3,8]]    = NoT(param_CaHSO42)              # CaHSO42        (3,8)
    [beta_0[3,9], beta_1[3,9], C_phi[3,9]]    = NoT(param_CaHS2)                # CaHS2          (3,9)
    [beta_0[3,10], beta_1[3,10], C_phi[3,10]] = NoT(param_CaI2)                 # CaI2           (3,10)
    [beta_0[3,11], beta_1[3,11], C_phi[3,11]] = TableA2_1(T,param_MgCl2)        # CaClO32        (3,11)
    [beta_0[3,12], beta_1[3,12], C_phi[3,12]] = TableA3to6(T,param_CaClO42)     # CaClO42        (3,12)
    [beta_0[3,13], beta_1[3,13], C_phi[3,13]] = TableA2_1(T,param_MgCl2)        # CaBrO32        (3,13)
    [beta_0[3,14], beta_1[3,14], C_phi[3,14]] = TableA2_1(T,param_MgCl2)        # CaCNS2         (3,14)
    [beta_0[3,15], beta_1[3,15], C_phi[3,15]] = TableA2_1(T,param_MgCl2)        # CaNO22         (3,15)
    [beta_0[3,16], beta_1[3,16], C_phi[3,16]] = TableA7(T,param_CaNO32)         # CaNO32         (3,16)
    [beta_0[3,17], beta_1[3,17], C_phi[3,17]] = TableA2_1(T,param_MgCl2)        # CaH2PO42       (3,17)
# N/A                                                                           # CaHPO4         (3,18)
# N/A                                                                           # Ca3(PO4)2      (3,19)
    [beta_0[3,20], beta_1[3,20], C_phi[3,20]] = TableA2_1(T,param_MgCl2)        # CaH2AsO42      (3,20)
# N/A                                                                           # CaHAsO4        (3,21)
# N/A                                                                           # Ca3AsO42       (3,22)
    [beta_0[3,23], beta_1[3,23], C_phi[3,23]] = NoT(param_CaHSO32)              # CaHSO32        (3,23)
    [beta_0[3,24], beta_1[3,24], C_phi[3,24]] = TableA1(T,param_CaSO4)          # CaSO3          (3,24)
# N/A                                                                           # CaAcetate      (3,25)

## Sr
    [beta_0[4,0], beta_1[4,0], C_phi[4,0]]    = TableA7(T,param_SrCl2)          # SrCl2          (4,0)
    [beta_0[4,1], beta_1[4,1], C_phi[4,1]]    = TableA1(T,param_SrSO4)          # SrSO4          (4,1)
# N/A                                                                           # SrCO3          (4,2)
    [beta_0[4,3], beta_1[4,3], C_phi[4,3]]    = NoT(param_SrHCO32)              # SrHCO32        (4,3)
    [beta_0[4,4], beta_1[4,4], C_phi[4,4]]    = TableA3to6(T,param_SrBr2)       # SrBr2          (4,4)
# N/A                                                                           # SrF2           (4,5)
    [beta_0[4,6], beta_1[4,6], C_phi[4,6]]    = TableA3to6(T,param_CaBOH42)     # SrB(OH)42      (4,6)
    [beta_0[4,7], beta_1[4,7], C_phi[4,7]]    = NoT(param_SrOH2)                # SrOH2          (4,7)
    [beta_0[4,8], beta_1[4,8], C_phi[4,8]]    = NoT(param_SrHSO42)              # SrHSO42        (4,8)
# N/A                                                                           # SrHS2          (4,9)
    [beta_0[4,10], beta_1[4,10], C_phi[4,10]] = NoT(param_SrI2)                 # SrI2           (4,10)
    [beta_0[4,11], beta_1[4,11], C_phi[4,11]] = TableA2_1(T,param_MgCl2)        # SrClO32        (4,11)
    [beta_0[4,12], beta_1[4,12], C_phi[4,12]] = TableA3to6(T,param_SrClO42)     # SrClO42        (4,12)
    [beta_0[4,13], beta_1[4,13], C_phi[4,13]] = TableA2_1(T,param_MgCl2)        # SrBrO32        (4,13)
    [beta_0[4,14], beta_1[4,14], C_phi[4,14]] = TableA2_1(T,param_MgCl2)        # SrCNS2         (4,14)
    [beta_0[4,15], beta_1[4,15], C_phi[4,15]] = TableA2_1(T,param_MgCl2)        # SrNO22         (4,15)
    [beta_0[4,16], beta_1[4,16], C_phi[4,16]] = TableA3to6(T,param_SrNO32)      # SrNO32         (4,16)
    [beta_0[4,17], beta_1[4,17], C_phi[4,17]] = TableA2_1(T,param_MgCl2)        # SrH2PO42       (4,17)
# N/A                                                                           # SrHPO4         (4,18)
# N/A                                                                           # Sr3(PO4)2      (4,19)
    [beta_0[4,20], beta_1[4,20], C_phi[4,20]] = TableA2_1(T,param_MgCl2)        # SrH2AsO42      (4,20)
# N/A                                                                           # SrHAsO4        (4,21)
# N/A                                                                           # Sr3(AsO4)2     (4,22)
    [beta_0[4,23], beta_1[4,23], C_phi[4,23]] = NoT(param_SrHSO32)              # SrHSO32        (4,23)
    [beta_0[4,24], beta_1[4,24], C_phi[4,24]] = TableA1(T,param_CaSO4)          # SrSO3          (4,24)
# N/A                                                                           # SrAcetate      (4,25)

## -----------------------------------------------------------------------------------------------------
## Minor Cation - Major Anion
## H
    [beta_0[5,0], beta_1[5,0], C_phi[5,0]]    = Eq_HCl(T,param_HCl)             # HCl            (5,0)
    [beta_0[5,1], beta_1[5,1], C_phi[5,1]]    = Eq_HSO4_C94(T,param_H_SO4_C94)  # H-SO4          (5,1)
# N/A                                                                           # H-HCO3         (5,2)
# N/A                                                                           # H-CO3          (5,3)
    [beta_0[5,4], beta_1[5,4], C_phi[5,4]]    = TableA7(T,param_HBr)            # HBr            (5,4)
# N/A                                                                           # HF             (5,5)
# N/A                                                                           # H-B(OH)4       (5,6)
    [C0_HSO4, C1_HSO4] = Eq_C0C1(T,H_SO4_C0C1_C94) 
    
##Li 
    [beta_0[6,0], beta_1[6,0], C_phi[6,0]]    = TableA7(T,param_LiCl)           # LiCl           (6,0)
    [beta_0[6,1], beta_1[6,1], C_phi[6,1]]    = TableA3to6(T,param_LiSO4)       # LiSO4          (6,1)
# N/A                                                                           # LiHCO3         (6,2)
# N/A                                                                           # LiCO3          (6,3)
    [beta_0[6,4], beta_1[6,4], C_phi[6,4]]    = TableA7(T,param_LiBr)           # LiBr           (6,4)
# N/A                                                                           # LiF            (6,5)
# N/A                                                                           # LiB(OH)4       (6,6)

## Rb 
    [beta_0[7,0], beta_1[7,0], C_phi[7,0]]    = TableA7(T,param_RbCl)           # RbCl           (7,0)
    [beta_0[7,1], beta_1[7,1], C_phi[7,1]]    = TableA3to6(T,param_RbSO4)       # RbSO4          (7,1)
# N/A                                                                           # RbHCO3         (7,2)
# N/A                                                                           # RbCO3          (7,3)
    [beta_0[7,4], beta_1[7,4], C_phi[7,4]]    = TableA7(T,param_RbBr)           # RbBr           (7,4)
    [beta_0[7,5], beta_1[7,5], C_phi[7,5]]    = TableA7(T,param_RbF)            # RbF            (7,5)
# N/A                                                                           # RbB(OH)4       (7,6) 

## Cs
    [beta_0[8,0], beta_1[8,0], C_phi[8,0]]    = TableA7(T,param_CsCl)           # CsCl           (8,0)
    [beta_0[8,1], beta_1[8,1], C_phi[8,1]]    = TableA3to6(T,param_CsSO4)       # CsSO4          (8,1)
# N/A                                                                           # CsHCO3         (8,2)
# N/A                                                                           # CsCO3          (8,3)
    [beta_0[8,4], beta_1[8,4], C_phi[8,4]]    = TableA7(T,param_CsBr)           # CsBr           (8,4)
    [beta_0[8,5], beta_1[8,5], C_phi[8,5]]    = TableA7(T,param_CsF)            # CsF            (8,5)
# N/A                                                                           # CsB(OH)4       (8,6)

## NH4
    [beta_0[9,0], beta_1[9,0], C_phi[9,0]] = TableA7(T,param_NH4Cl)             # NH4Cl          (9,0)
    [beta_0[9,1], beta_1[9,1], C_phi[9,1]] = NoT(param_SO4NH42)                 # SO4NH42        (9,1)
# N/A                                                                           # NH4HCO3        (9,2)
# N/A                                                                           # NH4CO3         (9,3)
    [beta_0[9,4], beta_1[9,4], C_phi[9,4]] = TableA7(T,param_NH4Br)             # NH4Br          (9,4)
    [beta_0[9,5], beta_1[9,5], C_phi[9,5]] = TableA7(T,param_NH4F)              # NH4F           (9,5)
# N/A                                                                           # NH4B(OH)4      (9,6)

## Ba
    [beta_0[10,0], beta_1[10,0], C_phi[10,0]] = TableA7(T,param_BaCl)           # BaCl           (10,0)
    [beta_0[10,1], beta_1[10,1], C_phi[10,1]] = NoT(param_BaSO4)                # BaSO4          (10,1)
    [beta_0[10,2], beta_1[10,2], C_phi[10,2]] = NoT(param_BaHCO3)               # BaHCO3         (10,2)
# N/A                                                                           # BaCO3          (10,3)
    [beta_0[10,4], beta_1[10,4], C_phi[10,4]] = TableA3to6(T,param_BaBr)        # BaBr           (10,4)
# N/A                                                                           # BaF            (10,5)
# N/A                                                                           # BaB(OH)4       (10,6)

## Mn
    [beta_0[11,0], beta_1[11,0], C_phi[11,0]] = TableA7(T,param_MnCl)           # MnCl           (11,0)
    [beta_0[11,1], beta_1[11,1], C_phi[11,1]] = NoT(param_MnSO4)                # MnSO4          (11,1)
# N/A                                                                           # MnHCO3         (11,2)
# N/A                                                                           # MnCO3          (11,3)
    [beta_0[11,4], beta_1[11,4], C_phi[11,4]] = NoT(param_MnBr)                 # MnBr           (11,4)
# N/A                                                                           # MnF            (11,5)
# N/A                                                                           # MnB(OH)4       (11,6)

## Fe(II)
    [beta_0[12,0], beta_1[12,0], C_phi[12,0]] = NoT(param_Fe2Cl)                # Fe2Cl          (12,0)
    [beta_0[12,1], beta_1[12,1], C_phi[12,1]] = NoT(param_Fe2SO4)               # Fe2SO4         (12,1)
# N/A                                                                           # Fe2HCO3        (12,2)
# N/A                                                                           # Fe2CO3         (12,3)
    [beta_0[12,4], beta_1[12,4], C_phi[12,4]] = NoT(param_Fe2Br)                # Fe2Br          (12,4)
# N/A                                                                           # Fe2F           (12,5)
# N/A                                                                           # Fe2B(OH)4      (12,6)

## Co
    [beta_0[13,0], beta_1[13,0], C_phi[13,0]] = TableA7(T,param_CoCl)           # CoCl           (13,0)
    [beta_0[13,1], beta_1[13,1], C_phi[13,1]] = NoT(param_CoSO4)                # CoSO4          (13,1)
# N/A                                                                           # CoHCO3         (13,2)
# N/A                                                                           # CoCO3          (13,3)
    [beta_0[13,4], beta_1[13,4], C_phi[13,4]] = NoT(param_CoBr)                 # CoBr           (13,4)
# N/A                                                                           # CoF            (13,5)
# N/A                                                                           # CoB(OH)4       (13,6)

## Ni(II)
    [beta_0[14,0], beta_1[14,0], C_phi[14,0]] = TableA7(T,param_NiCl)           # NiCl           (14,0)
    [beta_0[14,1], beta_1[14,1], C_phi[14,1]] = NoT(param_NiSO4)                # NiSO4          (14,1)
# N/A                                                                           # NiHCO3         (14,2)
# N/A                                                                           # NiCO3          (14,3)
    [beta_0[14,4], beta_1[14,4], C_phi[14,4]] = NoT(param_NiBr)                 # NiBr           (14,4)
# N/A                                                                           # NiF            (14,5)
# N/A                                                                           # NiB(OH)4       (14,6)

## Cu (II)
    [beta_0[15,0], beta_1[15,0], C_phi[15,0]] = TableA3to6(T,param_CuCl)        # CuCl           (15,0)
    [beta_0[15,1], beta_1[15,1], C_phi[15,1]] = TableA3to6(T,param_CuSO4)       # CuSO4          (15,1)
# N/A                                                                           # CuHCO3         (15,2)
# N/A                                                                           # CuCO3          (15,3)
    [beta_0[15,4], beta_1[15,4], C_phi[15,4]] = NoT(param_CuBr)                 # CuBr           (15,4)
# N/A                                                                           # CuF            (15,5)
# N/A                                                                           # CuB(OH)4       (15,6)

## Zn
    [beta_0[16,0], beta_1[16,0], C_phi[16,0]] = NoT(param_ZnCl)                 # ZnCl           (16,0)
    [beta_0[16,1], beta_1[16,1], C_phi[16,1]] = TableA3to6(T,param_ZnSO4)       # ZnSO4          (16,1)
# N/A                                                                           # ZnHCO3         (16,2)
# N/A                                                                           # ZnCO3          (16,3)
    [beta_0[16,4], beta_1[16,4], C_phi[16,4]] = NoT(param_ZnBr)                 # ZnBr           (16,4)
# N/A                                                                           # ZnF            (16,5)
# N/A                                                                           # ZnB(OH)4       (16,6)

## UO2
    [beta_0[17,0], beta_1[17,0], C_phi[17,0]] = NoT(param_UO2Cl)                # UO2Cl          (17,0)
    [beta_0[17,1], beta_1[17,1], C_phi[17,1]] = NoT(param_UO2SO4)               # UO2SO4         (17,1)
# N/A                                                                           # UO2HCO3        (17,2)
# N/A                                                                           # UO2CO3         (17,3)
    [beta_0[17,4], beta_1[17,4], C_phi[17,4]] = NoT(param_UO2Br)                # UO2Br          (17,4)
# N/A                                                                           # UO2F           (17,5)
# N/A                                                                           # UO2B(OH)4      (17,6)

## Be
# N/A                                                                           # BeCl           (18,0)
    [beta_0[18,1], beta_1[18,1], C_phi[18,1]] = NoT(param_BeSO4)                # BeSO4          (18,1)
# N/A                                                                           # BeHCO3-BeB(OH)4

## Cd
    [beta_0[19,0], beta_1[19,0], C_phi[19,0]] = NoT(param_CdCl)                 # CdCl           (19,0)
    [beta_0[19,1], beta_1[19,1], C_phi[19,1]] = TableA3to6(T,param_CdSO4)       # CdSO4          (19,1)
# N/A                                                                           # CdHCO3-CdB(OH)4

## Pb(II)
    [beta_0[20,0], beta_1[20,0], C_phi[20,0]] = NoT(param_PbCl)                 # PbCl           (20,0)
    [beta_0[20,1], beta_1[20,1], C_phi[20,1]] = TableA3to6(T,param_CdSO4)       # PbSO4          (20,1)
# N/A                                                                           # PbHCO3-PbB(OH)4 

## Cu(I)
    [beta_0[21,0], beta_1[21,0], C_phi[21,0]] = NoT(param_Cu1Cl)                # Cu1Cl          (21,0)
# N/A                                                                           # Cu1SO4 -> Cu1B(OH)4 

## -----------------------------------------------------------------------------------------------------
## REE - Cl (Interaction between REE and other major ions are N/A) 
## La
    [beta_0[22,0], beta_1[22,0], C_phi[22,0]] = Eq_REE (T,param_LaCl3)          # LaCl3          (22,0)
                                                    
## Ce
    [beta_0[23,0], beta_1[23,0], C_phi[23,0]] = Eq_REE (T,param_CeCl3)          # CeCl2          (23,0)
                                                    
## Pr
    [beta_0[24,0], beta_1[24,0], C_phi[24,0]] = Eq_REE(T,param_PrCl3)           # PrCl2          (24,0)
                                                    
## Nd
    [beta_0[25,0], beta_1[25,0], C_phi[25,0]] = Eq_REE (T,param_NdCl3)          # NdCl2          (25,0)
                                                    
## Pm
    [beta_0[26,0], beta_1[26,0], C_phi[26,0]] = Eq_REE (T,param_PmCl3)          # PmCl2          (26,0)
                                                    
## Sm
    [beta_0[27,0], beta_1[27,0], C_phi[27,0]] = Eq_REE (T,param_SmCl3)          # SmCl2          (27,0)
                                                    
## Eu
    [beta_0[28,0], beta_1[28,0], C_phi[28,0]] = Eq_REE(T,param_EuCl3)           # EuCl2          (28,0)
                                                    
## Gd 
    [beta_0[29,0], beta_1[29,0], C_phi[29,0]] = Eq_REE(T,param_GdCl3)           # GdCl2          (29,0)
                                                    
## Tb 
    [beta_0[30,0], beta_1[30,0], C_phi[30,0]] = Eq_REE(T,param_TbCl3)           # TbCl2          (30,0)
                                                    
## Dy 
    [beta_0[31,0], beta_1[31,0], C_phi[31,0]] = Eq_REE (T,param_DyCl3)          # DyCl2          (31,0)
                                                    
## Ho 
    [beta_0[32,0], beta_1[32,0], C_phi[32,0]] = Eq_REE (T,param_HoCl3)          # HoCl2          (32,0)
                                                    
## Er 
    [beta_0[33,0], beta_1[33,0], C_phi[33,0]] = Eq_REE (T,param_ErCl3)          # ErCl2          (33,0)
                                                    
## Tm 
    [beta_0[34,0], beta_1[34,0], C_phi[34,0]] = Eq_REE(T,param_TmCl3)           # LaCl2          (34,0)
                                                    
## Yb 
    [beta_0[35,0], beta_1[35,0], C_phi[35,0]] = Eq_REE (T,param_YbCl3)          # YbCl2          (35,0)
                                                    
## Lu
    [beta_0[36,0], beta_1[36,0], C_phi[36,0]] = Eq_REE(T,param_LuCl3)           # LuCl2          (36,0)
                                                    
## Y
    [beta_0[37,0], beta_1[37,0], C_phi[37,0]] = Eq_REE(T,param_YCl3)            # YCl2           (37,0)
                                                    
## Al   
    [beta_0[38,0], beta_1[38,0], C_phi[38,0]] = NoT(param_AlCl3)                # AlCl3          (38,0)
                                                    
## Ga 
    [beta_0[39,0], beta_1[39,0], C_phi[39,0]] = NoT(param_GaCl3)                # GaCl3          (39,0)
# param_GaClO4  = param_GaCl3;
                                                    
## In  
    [beta_0[40,0], beta_1[40,0], C_phi[40,0]] = NoT(param_InCl3)                # InCl3          (40,0)
                                                    
## Fe(III)
    [beta_0[41,0], beta_1[41,0], C_phi[41,0]] = TableA3to6(T,param_Fe3Cl3)      # Fe3Cl3         (41,0)
                                                    
## -----------------------------------------------------------------------------------------------------                                                    
## Beta 2 :- for 2:2 ion pairs beta_2 is needed. if not specified below
# no parameter is avaliable/set to zero

    beta_2[2,1]  = Eq_b2_MgSO4(T, beta2_MgSO4)                                 # MgSO4 
    beta_2[2,6]  = M_BOH4(T,beta2_MgBOH42)                                     # MgBOH42
    beta_2[3,1]  = Eq_b2_CaSO4(T,beta2_CaSO4)                                  # CaSO4
    beta_2[3,6]  = M_BOH4(T,beta2_CaBOH42)                                     # CaBOH42
    beta_2[4,1]  = beta_2[3,1]                                                 # SrSO4
    beta_2[4,6]  = beta_2[3,6]                                                 # SrBOH42
    beta_2[4,24] = beta_2[3,1]                                                 # SrSO3
    beta_2[2,24] = beta2_MgSO3                                                 # MgSO3
    beta_2[3,24] = beta_2[3,1]                                                 # CaSO3
    beta_2[10,1] = beta2_BaSO4                                                 # BaSO4
    beta_2[11,1] = beta2_MnSO4                                                 # MnSO4
    beta_2[12,1] = beta2_Fe2SO4                                                # Fe2SO4
    beta_2[13,1] = beta2_CoSO4                                                 # CoSO4
    beta_2[14,1] = beta2_NiSO4                                                 # NiSO4
    beta_2[15,1] = Eq_b2_XSO4(T,beta2_CuSO4)                                   # CuSO4
    beta_2[16,6] = Eq_b2_XSO4(T,beta2_ZnSO4)                                   # ZnSO4
    beta_2[18,1] = beta2_BeSO4                                                 # BeSO4
    beta_2[19,1] = Eq_b2_XSO4(T,beta2_CdSO4)                                   # CdSO4
    beta_2[20,1] = beta_2[19,1]                                                # PbSO4 
                                                    
    return beta_0, beta_1, beta_2, C_phi, C0_HSO4, C1_HSO4                                               

In [20]:
b0,b1,b2,Cphi,C0,C1 = binaryV3(25)
np.save('beta_0_T25.npy',b0)
print(b0)

[[-1.04419906e+01 -5.67174834e+01 -2.02702896e+00 ... -4.56057000e+01
  -6.33375600e+01  1.42600000e-01]
 [-1.85012170e+01 -3.74507973e+01 -5.52180705e-01 ...  0.00000000e+00
   0.00000000e+00  1.58700000e-01]
 [ 5.53145847e-01 -1.91983514e+02  0.00000000e+00 ...  3.50000000e-01
  -2.80000000e+00  0.00000000e+00]
 ...
 [ 8.25400000e-01  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.12000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.78928225e-01  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [21]:
b0,b1,b2,Cphi,C0,C1 = binaryV3(25)
np.save('beta_1_T25.npy',b1)
print(b1)

[[ 4.34555029e+00 -4.61355880e+02 -5.31727499e+00 ...  1.47446000e+02
   2.26696300e+02  3.23700000e-01]
 [ 9.27757042e+00  7.01075548e+01  1.78651210e+00 ...  0.00000000e+00
   0.00000000e+00  3.25100000e-01]
 [ 2.34401556e+00 -1.81897055e+02  0.00000000e+00 ...  1.22000000e+00
   1.29000000e+01  0.00000000e+00]
 ...
 [ 6.52930000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.56670000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.18165302e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [22]:
b0,b1,b2,Cphi,C0,C1 = binaryV3(25)
np.save('beta_2_T25.npy',b2)
print(b2)

[[    0.            0.            0.        ...     0.
      0.            0.       ]
 [    0.            0.            0.        ...     0.
      0.            0.       ]
 [    0.        -1810.5023583     0.        ...     0.
   -201.            0.       ]
 ...
 [    0.            0.            0.        ...     0.
      0.            0.       ]
 [    0.            0.            0.        ...     0.
      0.            0.       ]
 [    0.            0.            0.        ...     0.
      0.            0.       ]]


In [23]:
b0,b1,b2,Cphi,C0,C1 = binaryV3(25)
np.save('C_phi_T25.npy',Cphi)
print(Cphi)

[[ 2.82302349e-01  4.63724576e+01  5.20000000e-03 ...  1.01575000e+01
   1.34473000e+01 -6.29000000e-03]
 [ 2.20108661e+00 -1.88000000e-02  5.00000000e-04 ...  0.00000000e+00
   0.00000000e+00 -6.60000000e-03]
 [ 5.34344194e-02  7.27897125e+01  0.00000000e+00 ... -7.20000000e-02
   0.00000000e+00  0.00000000e+00]
 ...
 [ 3.48000000e-02  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.92839149e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [24]:
b0,b1,b2,Cphi,C0,C1 = binaryV3(25)
np.save('C0_HSO4_T25.npy',C0)
print(C0)

-3.594982012941059


In [25]:
b0,b1,b2,Cphi,C0,C1 = binaryV3(25)
np.save('C1_HSO4_T25.npy',C1)
print(C1)

-33.537455920041126


In [ ]:
a = [[0,0,0],[0,1,2]]
b = [[1,2,3],[4,5,6]]

print(a[1][1])
print(b[1][1])